In [54]:
from arcgis.gis import GIS, Item
import os
from multiprocessing import Process, Queue

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')

import reach_tools
import hydrology

gis = GIS(username=os.getenv('ARCGIS_USERNAME'), password=os.getenv('ARCGIS_PASSWORD'))

url_reach_line = os.getenv('URL_REACH_LINE')
url_reach_centroid = os.getenv('URL_REACH_CENTROID')
url_reach_points = os.getenv('URL_REACH_POINT')

lyr_reach_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_reach_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_reach_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
err_df = lyr_reach_centroid.query(where='error = 1', out_fields=['reach_id'], return_geometry=False).sdf
err_df.head()

,OBJECTID,reach_id
0,13,1
1,14,2
2,15,3
3,16,4
4,17,5


In [56]:
err_reach_id_str = '1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,689,892,1053,1098,1217,1229,1967,745,2028,2657,3010,3118,3119,3678,3777,3778,3779,3813,3879,3908,4140,4143,4207,4227,4352,4353,4389,4536,4581,4582,4583,4585,4586,4616,4625,4651,4654,4655,4656,4684,4692,4699,4700,4702,4704,4705,4706,4712,4715,4720,4721,4722,4723,4724,4727,4731,4732,4733,4734,4740,4742,4750,4753,4754,4756,4759,4768,4770,4781,4792,5001,5006,5048,5065,5067,5092,5095,5104,5105,5110,5121,5147,5149,5184,5185,5190,5212,5213,5833,1745,1856,2501,2542,2550,2551,2552,2553,2554,2560,2561,2562,2962,2972,3001,3113,3155,3156,3157,3158,3159,3162,3160,3161,3163,3164,3172,3173,3218,3219,3220,3221,3249,3250,3251,3252,3253,3254,3255,3256,3257,3258,3259,3260,3261,3262,3264,3265,3266,3267,3268,3269,3270,3271,3272,3273,3274,3276,3275,3277,3278,3279,3280,3281,3282,3423,3556,3557,3748,3870,3872,3884,3899,3933,3952,3957,3958,4004,4029,4131,4132,4149,4152,4158,4166,4211,4287,4317,4319,4357,4375,4406,4410,4417,4418,4419,4467,4577,4652,4653,4689,4693,4695,4696,4697,4698,4701,4703,4707,4708,4709,4710,4711,4713,4714,4716,4718,4719,4717,4725,4726,4728,4730,4729,4736,4735,4738,4737,4739,4741,4743,4744,4746,4745,4747,4748,4749,4751,4752,4755,4758,4757,4760,4762,4761,4763,4775,5111,5196,5207,5239,5240,5263,5286,5435,5507,5589,5600,5601,5626,5625,5628,5642,5643,5645,5672,5762,5763,5764,5765,5813,5818,5821,5822,5899,5900,5901,5967,5979,5990,5179,5297,5342,5405,5406,5408,5771,6037,6038,6243,6431,6446,6466,6467,6478,6561,6829,10174,10175,10242,10246,10248,10247,10250,10249,10251,10252,10266,10278,10289,10307,10308,10309,10310,10327,10343,10355,10396,10506,10591,10624,10646,10676,10844,10846,10854,10907,10931,10938,10991,11037,11041,11050'

In [57]:
err_reach_id_arr = err_df['reach_id'].unique()
print(len(err_reach_id_arr))
','.join(err_reach_id_arr)

381


'1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,689,892,1053,1098,1217,1229,1967,745,2028,2657,3010,3118,3119,3678,3777,3778,3779,3813,3879,3908,4140,4143,4207,4227,4352,4353,4389,4536,4581,4582,4583,4585,4586,4616,4625,4651,4654,4655,4656,4684,4692,4699,4700,4702,4704,4705,4706,4712,4715,4720,4721,4722,4723,4724,4727,4731,4732,4733,4734,4740,4742,4750,4753,4754,4756,4759,4768,4770,4781,4792,5001,5006,5048,5065,5067,5092,5095,5104,5105,5110,5121,5147,5149,5184,5185,5190,5212,5213,5833,1745,1856,2501,2542,2550,2551,2552,2553,2554,2560,2561,2562,2962,2972,3001,3113,3155,3156,3157,3158,3159,3162,3160,3161,3163,3164,3172,3173,3218,3219,3220,3221,3249,3250,3251,3252,3253,3254,3255,3256,3257,3258,3259,3260,3261,3262,3264,3265,3266,3267,3268,3269,3270,3271,3272,3273,3274,3276,3275,3277,3278,3279,3280,3281,3282,3423,3556,3557,3748,3870,3872,3884,3899,3933,3952,3957,3958,4004

In [61]:
where_clause = ' OR '.join([f"reach_id = '{reach_id}'" for reach_id in err_reach_id_arr])
where_clause

"reach_id = '1' OR reach_id = '2' OR reach_id = '3' OR reach_id = '4' OR reach_id = '5' OR reach_id = '6' OR reach_id = '7' OR reach_id = '8' OR reach_id = '9' OR reach_id = '10' OR reach_id = '11' OR reach_id = '12' OR reach_id = '13' OR reach_id = '14' OR reach_id = '15' OR reach_id = '16' OR reach_id = '17' OR reach_id = '18' OR reach_id = '19' OR reach_id = '20' OR reach_id = '21' OR reach_id = '22' OR reach_id = '23' OR reach_id = '24' OR reach_id = '25' OR reach_id = '26' OR reach_id = '27' OR reach_id = '28' OR reach_id = '29' OR reach_id = '30' OR reach_id = '31' OR reach_id = '32' OR reach_id = '33' OR reach_id = '34' OR reach_id = '35' OR reach_id = '36' OR reach_id = '37' OR reach_id = '38' OR reach_id = '39' OR reach_id = '40' OR reach_id = '41' OR reach_id = '42' OR reach_id = '43' OR reach_id = '45' OR reach_id = '46' OR reach_id = '47' OR reach_id = '48' OR reach_id = '49' OR reach_id = '50' OR reach_id = '51' OR reach_id = '52' OR reach_id = '53' OR reach_id = '54' OR r

In [71]:
where_clause = ' OR '.join([f"reach_id = '{reach_id}'" for reach_id in err_reach_id_arr[:330]])
df = lyr_reach_points.query(where=where_clause, out_fields=['reach_id'], return_geometry=False).sdf
df.head()

,OBJECTID,reach_id
0,25,1
1,26,1
2,27,2
3,28,2
4,29,3


In [46]:
','.join(df['OBJECTID'].astype(str))

'25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,1171,1172,1509,1510,1911,1912,1913,2114,2243,2366,2501,3556,3557,3778,3779,5051,5052,5133,5134,5151,5152,5153,5154,5155,5156,5157,5158,5159,5160,5167,5168,5169,5170,5171,5172,5961,5962,5985,5986,6041,6042,6265,6266,6347,6348,6351,6352,6355,6356,6357,6358,6359,6360,6363,6364,6365,6366,6367,6368,6369,6370,6371,6372,6375,6376,6379,6380,6465,6466,6467,6468,6469,6470,6471,6472,6531,6532,6533,6534,6535,6536,6537,6538,6539,6540,6541,6542,6543,6544,6545,6546,6547,6548,6549,6550,6551,6552,6553,6554,6555,6556,6557,6558,6559,6560,6561,6562,6563,6564,6565,6566,6567,6568,6569,6570,6571,6572,6573,6574,6575,65

In [53]:
err_df = lyr_reach_centroid.query(where='error = 1', out_fields=['reach_id'], return_geometry=False).sdf
err_reach_id_arr = err_df['reach_id'].unique()
where_clause = ' OR '.join([f"reach_id = '{reach_id}'" for reach_id in err_reach_id_arr])
fs_points = lyr_reach_points.query(where_clause)
fs_points


Unable to perform query. Please check your parameters.


RuntimeError: 
Unable to perform query. Please check your parameters.
(Error Code: 400)

In [ ]:
def purge_invalid():
    err_df = lyr_reach_centroid.query(where='error = 1', out_fields=['reach_id'], return_geometry=False).sdf
    err_reach_id_arr = err_df['reach_id'].unique()
    where_clause = ' OR '.join([f"reach_id = '{reach_id}'" for reach_id in err_reach_id_arr])
    